In [2]:
# !pip3 install osmnx
# !pip3 install networkx

import os, tqdm
import pandas as pd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points, transform
import pyproj

In [3]:
%%capture
# Configure OSMnx
ox.config(use_cache=True, log_console=True)

In [4]:
os.makedirs('./data', exist_ok=True)

In [5]:
loc = 'Padua, Italy'

- Compare walk network vs all

In [7]:
# Download the street network

# what type of street network to get if custom_filter is None
# network_type = {"all", "all_public", "bike", "drive", "drive_service", "walk"}

# network_type = 'walk'

city_graph_walk = ox.graph_from_place(loc, network_type='walk', simplify=True, retain_all=True, truncate_by_edge=True)
city_graph_all = ox.graph_from_place(loc, network_type='all', simplify=True, retain_all=True, truncate_by_edge=True)
city_graph_all_public = ox.graph_from_place(loc, network_type='all_public', simplify=True, retain_all=True, truncate_by_edge=True)

- Graph simplification
- Less than 10 seconds -> Merge?

In [7]:
%%capture
# https://wiki.openstreetmap.org/wiki/Key:amenity?uselang=en-GB

# shops = ox.geometries_from_place(loc, tags={'shop': True})
supermarkets = ox.geometries_from_place(loc, tags={'shop': ['supermarket','convenience']})
# hospitals = ox.geometries_from_place(loc, tags={'amenity': 'hospital'})
pharmacies = ox.geometries_from_place(loc, tags={'amenity': 'pharmacy'})
kindergarten = ox.geometries_from_place(loc, tags={'amenity': 'kindergarten'})
post_office = ox.geometries_from_place(loc, tags={'amenity': 'post_office'})
doctors = ox.geometries_from_place(loc, tags={'amenity': 'doctors'})
cafe = ox.geometries_from_place(loc, tags={'amenity': ['cafe', 'bar']})

In [48]:
def line_string_distance(line_string): # LineString(Point(lon, lat), Point(lon, lat))
    # Define a projection transformer (WGS84 to a planar coordinate system)
    proj_wgs84 = pyproj.CRS('EPSG:4326')
    proj_utm = pyproj.CRS('EPSG:32610')  # UTM zone 10N, adjust as needed for your location
    project = pyproj.Transformer.from_crs(proj_wgs84, proj_utm, always_xy=True).transform

    # Transform the LineString to the planar coordinate system
    line_transformed = transform(project, line_string)

    # Calculate the distance
    return line_transformed.length

In [49]:
# Define a function to insert a service node into the graph
def insert_service_node(graph, service_type, service_lon, service_lat):
    
    # Find the nearest edge and get the geometry to the service location
    u, v, key = ox.nearest_edges(graph, X=service_lon, Y=service_lat)

    # Create a point for the service location
    point = Point(service_lon, service_lat) # (lon, lat)

    # If edge is not a straight line, i.e. contains a 'geometry'
    try:
        edge_geom = graph.edges[u, v, key]
        # Check if (u, v have x, y or y, x coordinates)
        edge_weight = graph.edges[u, v, key]['length']
        edge_geom = edge_geom['geometry']

    except:
        # Coordinates of the nearest_edges
        edge_geom = LineString([(graph.nodes[u]['x'], graph.nodes[u]['y']),(graph.nodes[v]['x'], graph.nodes[v]['y'])])
        edge_weight = line_string_distance(edge_geom)
        
    # Calculate the closest point on the edge to the service location
    closest_point_on_edge = nearest_points(edge_geom, point)[0] # (lon. lat)

    # Create new edges with the split geometries
    edge_geom_1 = LineString([edge_geom.coords[0], (closest_point_on_edge.x, closest_point_on_edge.y)])
    edge_geom_2 = LineString([(closest_point_on_edge.x, closest_point_on_edge.y), edge_geom.coords[-1]])

    # Insert the new node into the graph
    new_node_id = max(graph.nodes()) + 1  # or any unique ID
    graph.add_node(new_node_id, x=closest_point_on_edge.x, y=closest_point_on_edge.y, service_type=service_type)

    # Ratio of first segment to total length
    # ratio_segment_1 =  point.distance(edge_geom.coords[0]) / edge_geom.length

    # Split weight based on ratio
    weight_segment_1 = line_string_distance(edge_geom_1)
    weight_segment_2 = line_string_distance(edge_geom_2)
    # weight_segment_1 = edge_weight * ratio_segment_1
    # weight_segment_2 = edge_weight * (1 - ratio_segment_1)

    # # Remove the old edge and add the new edges
    graph.remove_edge(u, v)
    graph.add_edge(u, new_node_id, key=key, geometry=edge_geom_1, length=weight_segment_1)
    graph.add_edge(new_node_id, v, key=key, geometry=edge_geom_2, length=weight_segment_2)

    return new_node_id

In [50]:
# x = longitude, y = latitude

city_graph2 = city_graph

for s in tqdm.tqdm(['supermarkets', 'pharmacies', 'kindergarten', 'post_office', 'doctors', 'cafe']):
    if 'node' in list(eval(s).index.get_level_values(0)):
        for v in eval(s)['geometry']['node'].get_coordinates().iterrows():
            insert_service_node(city_graph2, s, v[1]['x'], v[1]['y'])
    if 'way' in list(eval(s).index.get_level_values(0)):
        for v in eval(s)['geometry']['way'].get_coordinates().groupby('osmid').agg('mean').iterrows():
            insert_service_node(city_graph2, s, v[1]['x'], v[1]['y'])

100%|██████████| 6/6 [06:56<00:00, 69.45s/it] 


In [51]:
nodes_df = pd.DataFrame.from_dict(dict(city_graph.nodes(data=True)), orient='index')
edges_df = nx.to_pandas_edgelist(city_graph)

In [54]:
nodes_df

,y,x,highway,street_count,ref,service_type
141540485,45.425355,11.872216,crossing,4.0,NaN,NaN
141542464,45.414087,11.877584,NaN,3.0,NaN,NaN
141542473,45.413035,11.890344,NaN,3.0,NaN,NaN
141542517,45.413578,11.879962,NaN,1.0,NaN,NaN
141542774,45.415890,11.880340,NaN,3.0,NaN,NaN
...,...,...,...,...,...,...
11882653632,45.408641,11.934269,NaN,NaN,NaN,cafe
11882653633,45.409727,11.860299,NaN,NaN,NaN,cafe
11882653634,45.411123,11.874199,NaN,NaN,NaN,cafe
11882653635,45.408290,11.876908,NaN,NaN,NaN,cafe


In [33]:
edges_df

,source,target,est_width,ref,access,osmid,reversed,highway,lanes,maxspeed,length,geometry,area,bridge,width,junction,tunnel,oneway,name,service
0,141540485,254672751,NaN,NaN,NaN,23516562,False,path,NaN,NaN,5.674,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
1,141540485,197522982,NaN,NaN,NaN,41224006,False,tertiary,2,NaN,31.809,NaN,NaN,NaN,NaN,NaN,NaN,False,Via Annibale da Bassano,NaN
2,141540485,254672878,NaN,NaN,NaN,41224006,True,tertiary,2,NaN,18.533,NaN,NaN,NaN,NaN,NaN,NaN,False,Via Annibale da Bassano,NaN
3,141540485,2522360915,NaN,NaN,NaN,244968997,True,path,NaN,NaN,8.132,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
4,141542464,522059963,NaN,NaN,NaN,42046679,False,residential,NaN,NaN,57.846,NaN,NaN,NaN,NaN,NaN,NaN,False,Via Trieste,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59005,11882652709,11882564649,NaN,NaN,NaN,1280059949,True,footway,NaN,NaN,143.663,"LINESTRING (11.8450545 45.4276855, 11.8444835 ...",NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
59006,11882652721,768681977,NaN,NaN,NaN,61670457,False,path,NaN,NaN,149.911,"LINESTRING (11.845305 45.4275853, 11.8452919 4...",NaN,NaN,4.5,NaN,NaN,False,NaN,NaN
59007,11882652721,471609376,NaN,NaN,NaN,61670457,True,path,NaN,NaN,3.279,"LINESTRING (11.845305 45.4275853, 11.8453095 4...",NaN,NaN,4.5,NaN,NaN,False,NaN,NaN
59008,11882652721,8092964449,NaN,NaN,NaN,1280059944,False,footway,NaN,NaN,20.230,"LINESTRING (11.845305 45.4275853, 11.8453521 4...",NaN,NaN,NaN,NaN,NaN,False,NaN,NaN


In [55]:
nodes_df.to_csv(os.path.join('./data', 'nodes_tmp.csv'))